In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from keras.models import load_model
import pandas as pd 
import spacy
#nltk  Stemming
from nltk.stem import PorterStemmer
import time
MAINDIR = "./gdrive/My Drive/EA-test/"
from joblib import load
import sys
sys.path.append(MAINDIR)  #this is only necessary to use libraries not loaded in Google Colab
#to translate tweet into english
from googletrans import Translator
#fake agent to make multiple request and avoid request limit
from fake_useragent import UserAgent

Using TensorFlow backend.


In [0]:
language = "en_core_web_sm" #select language for spacy
nlp = spacy.load(language)
ps = PorterStemmer()


In [0]:
#generate fake agent
ua = UserAgent()

In [0]:
#Load file with tweets
df = pd.read_csv(MAINDIR + 'DFtweets.csv')

In [0]:
#Function to translate tweet into english
def translate(txt):
  translator = Translator(user_agent=ua.random)
  try:
    trsl = translator.translate(txt)
    time.sleep(3)
    return trsl.text
  except:
    print('FAIL')

In [0]:
df['text_en'] = df.text.apply(translate)

In [0]:
#df.to_csv(MAINDIR+'DFTweets_translated.csv', index=False) # save result
df = pd.read_csv(MAINDIR+'DFTweets_translated.csv') #load result

## Function to preprocess text

In [0]:
#to apply nlp spacy function
def nlp_(txt):
  if type(txt) == str:
    return nlp(txt)
  else:
    return 'xxxxxxxxxxxxxxxxx'

In [0]:
#Preprocess function 
#Exclude words:
wordEX = []
posEx = ['PUNCT','SPACE']
is_stopEx = []
lemmaEx = []
stemEx = []


def text_transf(texts, method = 'lemma_stem'):
  txt2 = []
  for w in texts:
    if (w.text.lower() in wordEX) | (w.pos_ in posEx) | (w.is_stop in is_stopEx) | (w.lemma_ in lemmaEx) | (ps.stem(w.text.lower()) in stemEx):
      continue
    if method == 'word':
      txt2.append(w.text.lower())
    elif method == 'lemma':
      txt2.append(w.lemma_.lower())
    elif method == 'stem':
      txt2.append(ps.stem(w.text.lower()))
    elif method == 'lemma_stem':
      txt2.append(ps.stem(w.lemma_))
      
  return txt2

In [0]:
#join list o result from text_transf in one string
def join_(txt):
  return ' '.join(txt)

## Preprocess text

In [0]:
df['preprocess'] = df.text_en.apply(nlp_)

In [0]:
#To delete any row if it fails with the nlp_ function
df = df[~(df.preprocess == 'xxxxxxxxxxxxxxxxx')]

In [0]:
df.loc[:,'preprocess'] = df['preprocess'].apply(text_transf)
df.loc[:,'preprocess'] = df['preprocess'].apply(join_)

In [0]:
#load tfidf_model
tfidf_model = load(MAINDIR + 'tfidf.pkl')
#transform text to numeric vector
X_vec = tfidf_model.transform(df.preprocess)
X_vec = X_vec.toarray()

In [0]:
#Load neural network model
model = load_model(MAINDIR +'NN_model.h5')

In [0]:
#classify vector x_vec
y_pre = model.predict(X_vec)

In [16]:
y_pre

array([[0.77603763, 0.22396243],
       [0.96732426, 0.03267571],
       [0.57380867, 0.42619136],
       ...,
       [0.49488866, 0.50511134],
       [0.8984796 , 0.10152045],
       [0.7272352 , 0.2727648 ]], dtype=float32)

In [0]:
#get prediction categories from y_pred
df['category_id'] = y_pre.argmax(axis=1)
#get positive probability
df['pos_prob'] = y_pre[:,1]

#Results table

In [18]:
#generate result table
group_location = df.groupby('location', as_index=False).agg({'text':'count','category_id':'sum'})
group_location.rename(columns={'text':'N_tweets','category_id':'N_pos'},inplace=True,)
group_location['N_neg'] = group_location.N_tweets - group_location.N_pos
group_location.sort_values('N_tweets',ascending=False)

,location,N_tweets,N_pos,N_neg
277,United Kingdom,13,2,11
243,Somewhere in Japan,9,1,8
97,France,9,5,4
254,"Tamil Nadu, India",8,2,6
59,"Chennai, India",7,0,7
73,Deutschland,6,3,3
121,India,6,0,6
321,"جدة, المملكة العربية السعودية",5,1,4
28,Belgium,4,0,4
83,España,4,3,1
